In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import imageio

## Simulation Functions

In [2]:
def getNeighbors(grid, point):
    neighbors = []
    for i in range(-1,2):
        for j in range(-1,2):
            try:
                if abs(i) != abs(j):
                    neighbors.append(grid[point[0] + i][point[1]+ j])
            except:
                neighbors.append(20)
            
    return neighbors

def calculateHeatTransient(cellTemp, neighborTemps, timeStep, thermalDiffusivity):
    cellTemp = cellTemp + timeStep*thermalDiffusivity*((neighborTemps[0] -2*cellTemp + neighborTemps[-1]) + 
                                                       (neighborTemps[1] -2*cellTemp + neighborTemps[-2]))
    return cellTemp

def heatTransferTransient(grid, timeStep):
    for i in range(0,len(grid)):
        for j in range(0,len(grid)):
            neighbors = getNeighbors(grid[:,:,0], (i,j))
            grid[:,:,0][i][j] = calculateHeatTransient(grid[:,:,0][i][j], neighbors, timeStep, grid[:,:,1][i][j])
              
    return grid

    

## Untility Functions

In [3]:
def makeGif(frames,name,minTemp,maxTemp):
    !mkdir frames
    
    counter=0
    images = []
    for i in range(0,len(frames)):
        plt.figure()
        plt.imshow(frames[i], cmap = "inferno", vmin = minTemp, vmax = maxTemp)
        plt.savefig("frames/" + str(counter)+ ".png")
        images.append(imageio.imread("frames/" + str(counter)+ ".png"))
        counter += 1
        plt.close()
        
    imageio.mimsave(name, images)
        
    !rm -r frames

## Welding Scenario

In [4]:
def initSheet(startTemp, thermalDiffusivity1, thermalDiffusivity2, resolution):
    
    imgTemp = np.full(resolution**2, startTemp)
    imgDiffusivity = np.full(resolution**2, thermalDiffusivity1)
    
    imgDiffusivity[:int(len(imgDiffusivity)/2)] = thermalDiffusivity2
    
    imgTemp = imgTemp.reshape((resolution,resolution)).T
    imgDiffusivity = imgDiffusivity.reshape((resolution,resolution)).T
    
    img = np.dstack([imgTemp, imgDiffusivity])
    
    return img

In [5]:
steelDiffusivity = 1.172*10**-3
brassDiffusivity = 3.42*10**-3

sheet = initSheet(20, steelDiffusivity, brassDiffusivity, 100)

frames = [sheet[:,:,0].copy()]
time = [0]
timeStep = 10
for i in range(0,len(sheet)):
    sheet[i,50,0] += 10000
    sheet = heatTransferTransient(sheet, timeStep)
    frames.append(sheet[:,:,0].copy())
    time.append(time[-1] + timeStep)

timeStep = 30
for i in range(0,700):
    sheet = heatTransferTransient(sheet, timeStep)
    frames.append(sheet[:,:,0].copy())
    time.append(time[-1] + timeStep)

In [6]:
makeGif(frames,"Welding.gif",20, 2000)

mkdir: frames: File exists


/var/folders/q1/jf3qx4ps6gjdmw1v4vfw_msc0000gn/T/ipykernel_8998/345337371.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread("frames/" + str(counter)+ ".png"))


## Burner Scenario 

In [7]:
def initCircle(center, radius, backgroundTemp, startTemp, backGroundDiffusivity, thermalDiffusivity, resolution):
    
    indexX,indexY = np.meshgrid(np.arange(0,resolution),np.arange(0,resolution))
    
    circle = (indexX - center[0])**2 + (indexY - center[1])**2
    mask = circle < radius
    
    imgTemp = np.full(len(mask.ravel()), backgroundTemp)
    imgDiffusivity = np.full(len(mask.ravel()), backGroundDiffusivity)
    
    imgTemp[mask.ravel()] = startTemp
    imgDiffusivity[mask.ravel()] = thermalDiffusivity
    
    imgTemp = imgTemp.reshape((resolution,resolution))
    imgDiffusivity = imgDiffusivity.reshape((resolution,resolution))
    
    img = np.dstack([imgTemp, imgDiffusivity])
    
    return img

def addHeat(center, radius, temp, resolution):
    
    indexX,indexY = np.meshgrid(np.arange(0,resolution),np.arange(0,resolution))
    
    circle = (indexX - center[0])**2 + (indexY - center[1])**2
    mask = circle < radius
    
    imgTemp = np.full(len(mask.ravel()), 0)
    
    imgTemp[mask.ravel()] = temp

    imgTemp = imgTemp.reshape((resolution,resolution))
    
    return imgTemp

In [8]:
steelDiffusivity = 1.172*10**-3
brassDiffusivity = 3.42*10**-3

img = initCircle((50,50), 500, 20, 20, brassDiffusivity, brassDiffusivity, 100)

frames = [img[:,:,0].copy()]

for i in range(0,100):
    img[:,:,0] += addHeat((50,50), 500, 10, 100)
    img = heatTransferTransient(img, 60)
    frames.append(img[:,:,0].copy())

for i in range(0,700):
    img = heatTransferTransient(img, 60)
    frames.append(img[:,:,0].copy())

In [ ]:
makeGif(frames,"Bruner.gif",20, 500)

/var/folders/q1/jf3qx4ps6gjdmw1v4vfw_msc0000gn/T/ipykernel_8998/345337371.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread("frames/" + str(counter)+ ".png"))
